# 財報爬蟲

http://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2017&SSEASON=3&REPORT_ID=C

合併報表優先（90%以上的財報都是合併財報），假如沒有合併報表，再抓個體財報

In [16]:
from finance_bot.utility import get_data_folder
import io

import pandas as pd
import requests
from fake_useragent import UserAgent


def get_financial_statements_path(stock_id, year, quarter):
    data_folder = get_data_folder()
    target_folder = data_folder / 'financial_statements' / stock_id
    target_folder.mkdir(parents=True, exist_ok=True)
    target_path = target_folder / f'{year}Q{quarter}.html'
    return target_path

def download_financial_statements(stock_id, year, quarter, report_type, dest_path):
    if year < 2013:
        raise ValueError('2013  (民國 102 年) 前不處理')
    if report_type not in ['A', 'B', 'C']:
        raise ValueError('不支援的 Report type (A：個別財報 / B：個體財報 / C：合併報表)')
    
    ua = UserAgent()
    res = requests.get(
        'https://mops.twse.com.tw/server-java/t164sb01',
        params={
            'step': 1,  # 不知啥用的
            'CO_ID': stock_id,
            'SYEAR': year,
            'SSEASON': quarter,
            'REPORT_ID': report_type,  # 個別財報(A) / 個體財報(B) / 合併報表(C)
        },
        headers={
            'user-agent': ua.random
        }
    )
    res.encoding = 'big5'
    body = res.text
    with dest_path.open('w', encoding='utf-8') as fp:
        fp.write(body)
    return body

def parse_financial_statement(year, report_type, source_path):
    with source_path.open('r', encoding='utf-8') as fp:
        body = fp.read()
        
    if year < 2019:
        dfs = pd.read_html(io.StringIO(body))
        if len(dfs) < 2:
            return None
        df = dfs[1]
        df = df.iloc[:, :2]
        df.columns = ['name', 'value']
        df['value'] = pd.to_numeric(df['value'], 'coerce')
        df = df.dropna()
        df = df.set_index('name')

        return {
            'share_capital': df['value'].loc['股本合計'],
        }
    else:
        try:
            dfs = pd.read_html(io.StringIO(body))
            df = dfs[0]
            df = df.iloc[:, :3]
            df.columns = ['code', 'name', 'value']
            df = df.set_index('code')
            df['value'] = pd.to_numeric(df['value'], 'coerce')
            df = df.dropna()
            return {
                'share_capital': df['value'].loc['3100'],
            }
        except ValueError:
            return None

stock_id = '1414'
year = 2019
quarter = 1
report_type = 'A'

path = get_financial_statements_path(stock_id, year, quarter)
body = download_financial_statements(stock_id, year, quarter, report_type, path)
data = parse_financial_statement(year, report_type, path)

print(data)

{'share_capital': 2200000.0}


In [1]:
import time
from finance_bot.infrastructure import infra
from sqlalchemy.orm import Session
from sqlalchemy import select
from finance_bot.model import TWStock

# 財報公布： 一般公司
# * 第一季（Q1）法說會：5/15 前
# * 第二季（Q2）財報：8/14 前
# * 第三季（Q3）財報：11/14 前
# * 第四季（Q4）財報及年報：隔年 3/31 前
# 
# 財報公布： 金融業
# * 第一季（Q1）財報：5/15 前
# * 第二季（Q2）財報：8/31 前
# * 第三季（Q3）財報：11/14 前

stock_id = '1101'

with Session(infra.db.engine) as session:
    date, = session.execute(
        select(TWStock.listing_date)
        .where(TWStock.stock_id == stock_id)
        .limit(1)
    ).first()
    
    start_date = max(pd.Timestamp(date), pd.Timestamp('2013'))
    periods = pd.period_range(
        pd.Period(start_date, freq='Q'),
        pd.Period(pd.Timestamp.now(), freq='Q') - 1,  # 上一季
        freq='Q',
    )

    financial_statements_list = []
    for period in periods:
        print(str(period) + ' ...')
        financial_statements = download_financial_statements(stock_id, period.year, period.quarter)
        financial_statements_list.append(financial_statements)
        time.sleep(30)
    
    df = pd.DataFrame([
        financial_statements 
        for financial_statements in financial_statements_list if financial_statements
    ])

df